# Installing Dependencies and Download Datasets

In [1]:
!pip install huggingface_hub jiwer evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.7 MB/s eta 0:00:00


In [2]:
!hf auth login

A new version of huggingface_hub (1.4.1) is available! You are using version 1.3.7.
To update, run: pip install -U huggingface_hub


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? [y/N]: y
Token is valid (permission: fineGrained).
The token `token` has been saved to /root/.cache/huggingface/s

In [3]:
!gdown 1Uw4lAW28xDh5tijF4on32CtvxsatzEPi

Downloading...
From (original): https://drive.google.com/uc?id=1Uw4lAW28xDh5tijF4on32CtvxsatzEPi
From (redirected): https://drive.google.com/uc?id=1Uw4lAW28xDh5tijF4on32CtvxsatzEPi&confirm=t&uuid=8dbb841f-41e7-4a35-a5d9-0973c3df28d3
To: /content/LibriSpeech_final.zip
100% 2.45G/2.45G [00:43<00:00, 56.1MB/s]


In [4]:
!unzip LibriSpeech_final.zip

Output streaming akan dipotong hingga 5000 baris terakhir.
   creating: LibriSpeech_final/train-clean/4406/
   creating: LibriSpeech_final/train-clean/4406/16882/
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0054.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0005.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0011.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/16882.trans.txt  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0001.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0068.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0052.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0006.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0003.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0039.wav  
  inflating: LibriSpeech_final/train-clean/4406/16882/4406-16882-0002.wav  
  inf

In [5]:
!pip install g2p_en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.4 MB/s eta 0:00:00
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=1e5849b702853040cb8ffe6ba54b13691a5d0a0df225e649172c9072bbd21516
  Stored in directory: /root/.cache/pip/wheels/24/a8/58/407063d8e5c1d4dd6594c99d12baa0108570b56a92325587dd
Successfully built distance


# Import Library

In [33]:
from datasets import load_dataset, Audio
from transformers import pipeline, Wav2Vec2Processor, Wav2Vec2ForCTC
import evaluate
import torch
import re
import librosa
import os
from tqdm import tqdm
import numpy as np

In [7]:
from g2p_en import G2p
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

# Config Device, Model and Evaluation Metric

In [8]:
cer_metric = evaluate.load("cer")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
device_id = 0 if torch.cuda.is_available() else -1
device_name = "cuda" if device_id == 0 else "cpu"

print(f"Using: {device_name}")

model_id = 'marx90/lingulu_wav2vec2_pronounciation_finetune'

Using: cuda


In [13]:
pipe = pipeline("automatic-speech-recognition", model=model_id, device=device_id)

Loading weights:   0%|          | 0/424 [00:00<?, ?it/s]

In [14]:
processor = Wav2Vec2Processor.from_pretrained("marx90/lingulu_wav2vec2_pronounciation_finetune")
model = Wav2Vec2ForCTC.from_pretrained("marx90/lingulu_wav2vec2_pronounciation_finetune")
model.eval()
model.to(device_name)

Loading weights:   0%|          | 0/424 [00:00<?, ?it/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

# Utility Function

In [45]:
g2p = G2p()

ARPABET_TO_IPA = {
    # Vowels (Monophthongs)
    'AA': 'ɑ', 'AA0': 'ɑ', 'AA1': 'ɑ', 'AA2': 'ɑ',       # bot
    'AE': 'æ', 'AE0': 'æ', 'AE1': 'æ', 'AE2': 'æ',       # bat
    'AH': 'ʌ', 'AH0': 'ə', 'AH1': 'ʌ', 'AH2': 'ʌ',       # butt (stressed vs. unstressed)
    'AO': 'ɔ', 'AO0': 'ɔ', 'AO1': 'ɔ', 'AO2': 'ɔ',       # bought
    'EH': 'ɛ', 'EH0': 'ɛ', 'EH1': 'ɛ', 'EH2': 'ɛ',       # bet
    'ER': 'ɝ', 'ER0': 'ɚ', 'ER1': 'ɝ', 'ER2': 'ɝ',       # bird (stressed vs. unstressed)
    'IH': 'ɪ', 'IH0': 'ɪ', 'IH1': 'ɪ', 'IH2': 'ɪ',       # bit
    'IY': 'i', 'IY0': 'i', 'IY1': 'i', 'IY2': 'i',       # beat
    'UH': 'ʊ', 'UH0': 'ʊ', 'UH1': 'ʊ', 'UH2': 'ʊ',       # book
    'UW': 'u', 'UW0': 'u', 'UW1': 'u', 'UW2': 'u',       # boot

    # Vowels (Diphthongs)
    'AW': 'aʊ', 'AW0': 'aʊ', 'AW1': 'aʊ', 'AW2': 'aʊ',     # bout
    'AY': 'aɪ', 'AY0': 'aɪ', 'AY1': 'aɪ', 'AY2': 'aɪ',     # bite
    'EY': 'eɪ', 'EY0': 'eɪ', 'EY1': 'eɪ', 'EY2': 'eɪ',     # bait
    'OW': 'oʊ', 'OW0': 'oʊ', 'OW1': 'oʊ', 'OW2': 'oʊ',     # boat
    'OY': 'ɔɪ', 'OY0': 'ɔɪ', 'OY1': 'ɔɪ', 'OY2': 'ɔɪ',     # boy

    # Consonants
    'P': 'p', 'B': 'b', 'T': 't', 'D': 'd', 'K': 'k', 'G': 'g',
    'CH': 'tʃ', 'JH': 'dʒ', 'F': 'f', 'V': 'v', 'TH': 'θ', 'DH': 'ð',
    'S': 's', 'Z': 'z', 'SH': 'ʃ', 'ZH': 'ʒ', 'HH': 'h', 'M': 'm',
    'N': 'n', 'NG': 'ŋ', 'L': 'l', 'R': 'ɹ', 'W': 'w', 'Y': 'j',

    # Syllabic Consonants (less common, but useful for completeness)
    'EM': 'm̩', 'EN': 'n̩', 'EL': 'l̩', 'NX': 'ɾ̃',
}

def arpabet_to_ipa_phonemes(arpabet_list):
    """
    Loads an audio file and performs a forward pass through the model to extract the raw logits.

    Args:
        audio_path (str): The file path to the audio (.wav) file.

    Returns:
        numpy.ndarray: The output logits from the model representing phoneme probabilities.
    """
    ipa = []
    for p in arpabet_list:
        ipa_symbol = ARPABET_TO_IPA.get(p)
        if ipa_symbol:
            ipa.append(ipa_symbol)
    return ipa

def transcribe_logits(audio_path):
    """
    Loads an audio file and performs a forward pass through the model to extract the raw logits.

    Args:
        audio_path (str): The file path to the audio (.wav) file.

    Returns:
        numpy.ndarray: The output logits from the model representing phoneme probabilities.
    """
    speech, sr = librosa.load(audio_path, sr=16_000)

    inputs = processor(
        speech,
        sampling_rate=16_000,
        return_tensors="pt",
        padding=True
    )
    inputs = {k: v.to(device_name) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits

    return logits.cpu().numpy()

def load_librispeech_dataset(root_dir):
    """
    Walks through a directory to load LibriSpeech samples, transcribing text into IPA phonemes.

    Args:
        root_dir (str): The root directory containing LibriSpeech .wav and .trans.txt files.

    Returns:
        list: A list of dictionaries containing the audio path, original text, and IPA phoneme sequence.
    """
    samples = []

    for root, _, files in os.walk(root_dir):
        trans_files = [f for f in files if f.endswith(".trans.txt")]
        if not trans_files:
            continue

        trans_path = os.path.join(root, trans_files[0])

        with open(trans_path, "r", encoding="utf-8") as f:
            transcripts = {}
            for line in f:
                utt_id, text = line.strip().split(" ", 1)
                transcripts[utt_id] = text.lower()

        for wav in files:
            if wav.endswith(".wav"):
                utt_id = wav.replace(".wav", "")
                if utt_id in transcripts:
                    arpabet_phonemes = [p for p in g2p(transcripts[utt_id]) if p.strip() and p not in [' ', ',', '.', '!', '?']]
                    ipa_phonemes = arpabet_to_ipa_phonemes(arpabet_phonemes)
                    samples.append({
                        "audio_path": os.path.join(root, wav),
                        "text": transcripts[utt_id],
                        "phonemes": "".join(ipa_phonemes)
                    })

    return samples

def predict_test_sample(batch):
    """
    Processes a single batch sample to generate a prediction and a phonemic reference for evaluation.

    Args:
        batch (dict): A dictionary containing audio data and ground truth text.

    Returns:
        dict: The updated batch dictionary with 'prediction' (text) and 'reference' (IPA phonemes).
    """
    inputs = processor(
        batch["audio"]["array"],
        sampling_rate=16_000,
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to(device_name) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits

    pred_ids = torch.argmax(logits, dim=-1)
    prediction = processor.batch_decode(pred_ids)[0]

    prediction = pipe(batch["audio"]["array"])['text']
    arpabet_phonemes = [p for p in g2p(batch['text']) if p.strip() and p not in [' ', ',', '.', '!', '?']]
    ipa_phonemes = arpabet_to_ipa_phonemes(arpabet_phonemes)

    batch["prediction"] = prediction
    batch["reference"] = "".join(ipa_phonemes)

    return batch

def encode_labels(text):
    """
    Tokenizes a string of text into input IDs suitable for the model's tokenizer.

    Args:
        text (str): The input text or phoneme string to encode.

    Returns:
        numpy.ndarray: The encoded input IDs.
    """
    return processor.tokenizer(
        text,
        return_tensors="np",
        padding=False
    ).input_ids[0]

def evaluate_train_eval_sample(dataset):
    """
    Evaluates the model performance on a given dataset using the Character Error Rate (CER).

    Args:
        dataset (list): A list of samples containing audio paths and target phonemes.

    Returns:
        float: The computed Character Error Rate (CER) for the dataset.
    """
    pred_strs = []
    label_strs = []

    for sample in tqdm(dataset):
        logits = transcribe_logits(sample["audio_path"])
        pred_ids = np.argmax(logits, axis=-1)

        pred_str = processor.batch_decode(pred_ids)[0]
        pred_strs.append(pred_str)

        label_ids = processor.tokenizer(
            sample["phonemes"],
            return_tensors="np"
        ).input_ids

        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        label_str = processor.batch_decode(
            label_ids,
            group_tokens=False
        )[0]

        label_strs.append(label_str)

    return cer_metric.compute(
        predictions=pred_strs,
        references=label_strs
    )

# Load Datasets

In [16]:
TRAIN_DATASET_PATH = "LibriSpeech_final/train-clean"
EVAL_DATASET_PATH = "LibriSpeech_final/test-clean"

train_dataset = load_librispeech_dataset(TRAIN_DATASET_PATH)
eval_dataset = load_librispeech_dataset(EVAL_DATASET_PATH)

In [17]:
test_dataset = load_dataset('KoelLabs/L2Arctic')
test_dataset = test_dataset['scripted']

print(test_dataset)

README.md:   0%|          | 0.00/3.91k [00:00<?, ?B/s]

data/spontaneous-00000-of-00001.parquet:   0%|          | 0.00/50.0M [00:00<?, ?B/s]

data/scripted-00000-of-00001.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

Generating spontaneous split:   0%|          | 0/22 [00:00<?, ? examples/s]

Generating scripted split:   0%|          | 0/3599 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'ipa', 'text', 'speaker_code', 'speaker_gender', 'speaker_native_language'],
    num_rows: 3599
})


In [18]:
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

# Evaluation

In [21]:
train_cer_score = evaluate_train_eval_sample(train_dataset)
print(f"Training Dataset CER: {train_cer_score:.4f}")

100%|██████████| 7051/7051 [22:37<00:00,  5.20it/s]


Training Dataset CER: 0.0261


In [46]:
eval_cer_score = evaluate_train_eval_sample(eval_dataset)
print(f"\nEvaluation Dataset CER: {eval_cer_score:.4f}")

100%|██████████| 453/453 [00:57<00:00,  7.88it/s]



Evaluation Dataset CER: 0.0337


In [43]:
test_results = test_dataset.map(predict_test_sample)

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

In [44]:
predictions = test_results["prediction"]
references = test_results["reference"]

test_cer_score = cer_metric.compute(predictions=predictions, references=references)
print(f"Testing Dataset CER: {test_cer_score:.4f}")

Testing Dataset CER: 0.0945
